## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
--select count(*) from bookings;
--select count(*) from facilities;
--select count(*) from members;
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select * from facilities where membercost != 0;


facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select distinct(name) from facilities where membercost =0;

name
Pool Table
Badminton Court
Table Tennis
Snooker Table


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql

select facid, name, membercost, monthlymaintenance from facilities where (membercost < (monthlymaintenance * 0.2)) and (membercost > 0)  ;


facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities where facid in (1,5);

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
select *, CASE
            WHEN monthlymaintenance > 100
               THEN 'Expensive'
               ELSE 'Cheap'
       END as expensecode
from facilities;

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,expensecode
0,Tennis Court 1,5.0,25.0,10000,200,Expensive
1,Tennis Court 2,5.0,25.0,8000,200,Expensive
2,Badminton Court,0.0,15.5,4000,50,Cheap
3,Table Tennis,0.0,5.0,320,10,Cheap
4,Massage Room 1,9.9,80.0,4000,3000,Expensive
5,Massage Room 2,9.9,80.0,4000,3000,Expensive
6,Squash Court,3.5,17.5,5000,80,Cheap
7,Snooker Table,0.0,5.0,450,15,Cheap
8,Pool Table,0.0,5.0,400,15,Cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
--select * from members;
select firstname, surname from members where firstname != 'GUEST' and surname != 'GUEST' ORDER BY (joindate) DESC


firstname,surname
Darren,Smith
Erica,Crumpet
John,Hunt
Hyacinth,Tupperware
Millicent,Purview
Henry,Worthington-Smyth
David,Farrell
Henrietta,Rumney
Douglas,Jones
Ramnaresh,Sarwin


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
--select * from facilities limit 10;
--select distinct(memid) from bookings where facid in (0,1);
--select * from members limit 10;

select distinct(m.memid), m.* from members m, bookings b where m.memid = b.memid and b.facid in (0,1) and m.firstname != 'GUEST' and m.surname != 'GUEST'

memid,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,null,2012-07-03T09:32:15.000+0000
16,16,Baker,Timothy,"329 James Street, Reading",58393,833-941-0824,13,2012-08-15T10:34:25.000+0000
22,22,Coplin,Joan,"85 Bard Street, Bloomington, Boston",43533,(822) 499-2232,16,2012-08-29T08:32:41.000+0000
10,10,Owen,Charles,"52 Cheshire Grove, Winchester, 28563",28563,(855) 542-5251,1,2012-08-03T19:42:37.000+0000
15,15,Bader,Florence,"264 Ursula Drive, Westford",84923,(833) 499-3527,9,2012-08-10T17:52:03.000+0000
24,24,Sarwin,Ramnaresh,"12 Bullington Lane, Boston",65464,(822) 413-1470,15,2012-09-01T08:44:42.000+0000
17,17,Pinker,David,"5 Impreza Road, Boston",65332,811 409-6734,13,2012-08-16T11:32:47.000+0000
36,36,Crumpet,Erica,"Crimson Road, North Reading",75655,(811) 732-4816,2,2012-09-22T08:36:38.000+0000
35,35,Hunt,John,"5 Bullington Lane, Boston",54333,(899) 720-6978,30,2012-09-19T11:32:45.000+0000
5,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09T10:44:09.000+0000


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
--select * from members
-- select * from facilities;
--select * from bookings;
-- Check with DJ if there is a way to do this without union

select b.memid, m.memid, CONCAT (f.name, ' --- ', m.surname, ', ', m.firstname) AS facility_name_mem_name, f.membercost, f.guestcost,  b.slots, b.starttime
from bookings b, facilities f, members m 
where 
  to_date(b.starttime) =  to_date('2012-09-14') and
  b.facid = f.facid and
  b.memid = m.memid and
  m.memid != 0 and 
  (f.membercost * b.slots > 30)
UNION
select b.memid, m.memid, CONCAT (f.name, ' --- ', m.surname, ', ', m.firstname) AS facility_name_mem_name, f.membercost, f.guestcost, b.slots, b.starttime
from bookings b, facilities f, members m 
where 
  to_date(b.starttime) =  to_date('2012-09-14') and
  b.facid = f.facid and
  b.memid = m.memid and
  m.memid = 0 and
  (f.guestcost * b.slots > 30);



memid,memid,facility_name_mem_name,membercost,guestcost,slots,starttime
13,13,"Massage Room 1 --- Farrell, Jemima",9.9,80.0,4,2012-09-14T14:00:00.000+0000
0,0,"Squash Court --- GUEST, GUEST",3.5,17.5,2,2012-09-14T15:00:00.000+0000
0,0,"Massage Room 1 --- GUEST, GUEST",9.9,80.0,2,2012-09-14T09:00:00.000+0000
0,0,"Massage Room 1 --- GUEST, GUEST",9.9,80.0,2,2012-09-14T16:00:00.000+0000
0,0,"Tennis Court 2 --- GUEST, GUEST",5.0,25.0,3,2012-09-14T14:00:00.000+0000
0,0,"Tennis Court 1 --- GUEST, GUEST",5.0,25.0,3,2012-09-14T16:00:00.000+0000
0,0,"Squash Court --- GUEST, GUEST",3.5,17.5,4,2012-09-14T09:30:00.000+0000
0,0,"Massage Room 2 --- GUEST, GUEST",9.9,80.0,4,2012-09-14T11:00:00.000+0000
0,0,"Tennis Court 2 --- GUEST, GUEST",5.0,25.0,6,2012-09-14T17:00:00.000+0000
0,0,"Massage Room 1 --- GUEST, GUEST",9.9,80.0,2,2012-09-14T13:00:00.000+0000


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql

select b.memid, m.memid, f.facid, b.facid, CONCAT (f.name, ' --- ', m.surname, ', ', m.firstname) AS facility_name_mem_name, f.membercost, f.guestcost,  b.slots, b.starttime
from bookings b, facilities f, members m 
where 
  b.bookid in (select bookid from bookings where to_date(starttime) =  to_date('2012-09-14')) and
  b.facid = f.facid and
  b.memid = m.memid and
  m.memid != 0 and 
  (f.membercost * b.slots > 30)
UNION
select b.memid, m.memid, f.facid, b.facid, CONCAT (f.name, ' --- ', m.surname, ', ', m.firstname) AS facility_name_mem_name, f.membercost, f.guestcost, b.slots, b.starttime
from bookings b, facilities f, members m 
where 
  b.bookid in (select bookid from bookings where to_date(starttime) =  to_date('2012-09-14')) and
  b.facid = f.facid and
  b.memid = m.memid and
  m.memid = 0 and
  (f.guestcost * b.slots > 30);
  


memid,memid,facid,facid,facility_name_mem_name,membercost,guestcost,slots,starttime
13,13,4,4,"Massage Room 1 --- Farrell, Jemima",9.9,80.0,4,2012-09-14T14:00:00.000+0000
0,0,1,1,"Tennis Court 2 --- GUEST, GUEST",5.0,25.0,3,2012-09-14T14:00:00.000+0000
0,0,4,4,"Massage Room 1 --- GUEST, GUEST",9.9,80.0,2,2012-09-14T09:00:00.000+0000
0,0,6,6,"Squash Court --- GUEST, GUEST",3.5,17.5,2,2012-09-14T15:00:00.000+0000
0,0,6,6,"Squash Court --- GUEST, GUEST",3.5,17.5,2,2012-09-14T12:30:00.000+0000
0,0,0,0,"Tennis Court 1 --- GUEST, GUEST",5.0,25.0,3,2012-09-14T16:00:00.000+0000
0,0,1,1,"Tennis Court 2 --- GUEST, GUEST",5.0,25.0,6,2012-09-14T17:00:00.000+0000
0,0,0,0,"Tennis Court 1 --- GUEST, GUEST",5.0,25.0,3,2012-09-14T19:00:00.000+0000
0,0,4,4,"Massage Room 1 --- GUEST, GUEST",9.9,80.0,2,2012-09-14T16:00:00.000+0000
0,0,4,4,"Massage Room 1 --- GUEST, GUEST",9.9,80.0,2,2012-09-14T13:00:00.000+0000


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
--select * from bookings;
--select * from facilities;
select f.name, SUM(CASE WHEN b.memid = 0 THEN b.slots * f.guestcost ELSE b.slots * f.membercost END) AS revenue 
from facilities f, bookings b 
where 
    f.facid = b.facid 
group by f.name
having revenue < 1000
sort by revenue;



name,revenue
Table Tennis,180.0
Snooker Table,240.0
Pool Table,270.0
